# Simon's world Cup 2018 Playground

This is the playground of mine to play with world cup 2018's data. The data is based on [this free API](http://worldcup.sfg.io/). This notebook is the testfield and lab where I can play, explore and learn from this project of mine, so please bear with me if it's too messy.

In [4]:
import json
import urllib.request

group_dict = {}
team_dict = {}
match_dict = {}

### classes of the project

In [5]:
class wc_group(object):
    def __init__(self, group_id, letter, teams):
        self.group_id = group_id
        self.letter = letter
        self.teams = teams
        
    def __repr__(self):
        return "Group: " + self.letter
    
    def teamInGroup(self):
        for team in self.teams.values():
            yield team
        
class wc_team(object):
    match_dict = {}
    completed_match_dict = {}
    def __init__(self, team_id, country, fifa_code):
        self.team_id = team_id
        self.country = country
        self.fifa_code = fifa_code
        
    def __repr__(self):
        return self.country + "(" + self.fifa_code + ")"
    
    def _addMatch(self, match):
        if self.country in match.getTeams():
            if match.getMatchId() not in self.match_dict:
                self.match_dict[match.getMatchId()] = match
                if match.isCompleted():
                    if match.getMatchId not in self.match_dict:
                        self.completed_match_dict[match.getMatchId()] = match
                
    def getFifaCode(self):
        return self.fifa_code
    
    def getCountryNameWithCode(self):
        return self.country + "(" + self.fifa_code + ")"
    
class wc_match(object):
    def __init__(self, location, venue, status , time, fifa_id, datetime ,home_team, away_team, home_event_list, away_event_list, winner):
        self.location = location
        self.venue = venue
        self.status = status
        self.time = time
        self.fifa_id = fifa_id
        self.datetime = datetime
        self.home_team = home_team
        self.away_team = away_team
        self.home_event_list = home_event_list
        self.away_event_list = away_event_list
        if self.status=='completed':
            if winner != 'Draw':
                self.winner = self.home_team if self.home_team.getFifaCode() == winner else self.away_team
            else:
                self.winner = None
        
    def __repr__(self):
        return "Match " + self.fifa_id + ": " + self.home_team.getCountryNameWithCode() + " v.s. " + self.away_team.getCountryNameWithCode() +\
               " on " + self.datetime + " in " + self.venue
    
    def getMatchId(self):
        return self.fifa_id
    
    def isCompleted(self):
        return True if self.status == 'completed' else False


### build the dict of group and the dict of team

In [7]:
group_results_json = urllib.request.urlopen('http://worldcup.sfg.io/teams/group_results')
group_results_python = json.loads(group_results_json.read())

for group in group_results_python:
    team_dict_in_group = {}
    for team in group['group']['teams']:
        team_dict_in_group[team['team']['country']] = wc_team(team['team']['id'], team['team']['country'], team['team']['fifa_code'])
        team_dict[team['team']['country']] = team_dict_in_group[team['team']['country']]
    group_dict[group['group']['letter']] = wc_group(group['group']['id'], group['group']['letter'], team_dict_in_group)

In [8]:
# print them
for g in group_dict.values():
    print(g)
    for t in g.teamInGroup():
        print(t)

Group: A
Russia(RUS)
Uruguay(URU)
Egypt(EGY)
Saudi Arabia(KSA)
Group: B
Iran(IRN)
Spain(ESP)
Portugal(POR)
Morocco(MAR)
Group: C
France(FRA)
Denmark(DEN)
Australia(AUS)
Peru(PER)
Group: D
Croatia(CRO)
Iceland(ISL)
Argentina(ARG)
Nigeria(NGA)
Group: E
Serbia(SRB)
Switzerland(SUI)
Brazil(BRA)
Costa Rica(CRC)
Group: F
Sweden(SWE)
Mexico(MEX)
Korea Republic(KOR)
Germany(GER)
Group: G
Belgium(BEL)
England(ENG)
Tunisia(TUN)
Panama(PAN)
Group: H
Japan(JPN)
Senegal(SEN)
Poland(POL)
Colombia(COL)


### build all matches in group A

In [9]:
g = group_dict['A']
for t in g.teamInGroup():
    f = urllib.request.urlopen("http://worldcup.sfg.io/matches/country?fifa_code=" + t.getFifaCode())
    match_list_for_this_team =  json.loads(f.read())
    for m in match_list_for_this_team:
        if not m['fifa_id'] in match_dict:
            match_dict[m['fifa_id']] = wc_match(location = m['venue'], venue = m['location'], status = m['status'], time = m['time'],
                                        fifa_id = m['fifa_id'], datetime = m['datetime'], home_team = team_dict[m['home_team']['country']],
                                        away_team = team_dict[m['away_team']['country']], home_event_list = m['home_team_events'],
                                        away_event_list = m['away_team_events'], winner = m['winner_code'])

In [10]:
# print the fetched match data so far (all group A matches)
for m_id, match in match_dict.items():
    print(match)

Match 300331516: Uruguay(URU) v.s. Russia(RUS) on 2018-06-25T14:00:00Z in Samara Arena
Match 300331495: Russia(RUS) v.s. Egypt(EGY) on 2018-06-19T18:00:00Z in Saint Petersburg Stadium
Match 300331503: Russia(RUS) v.s. Saudi Arabia(KSA) on 2018-06-14T15:00:00Z in Luzhniki Stadium
Match 300331530: Uruguay(URU) v.s. Saudi Arabia(KSA) on 2018-06-20T15:00:00Z in Rostov Arena
Match 300353632: Egypt(EGY) v.s. Uruguay(URU) on 2018-06-15T12:00:00Z in Ekaterinburg Arena
Match 300331509: Saudi Arabia(KSA) v.s. Egypt(EGY) on 2018-06-25T14:00:00Z in Volgograd Arena
